In [6]:
import numpy as np
import numpy.random as rand

In [7]:
class Cell():
    target = 0
    cell_type = ""
    prob_not_found = 0
    def __init__(self, cell_type, prob_not_found):
        self.cell_type = cell_type
        self.prob_not_found = prob_not_found
        

In [8]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")
    
env = [[] for i in range(dimension)] # create a list of empty lists
cell_type = ""
prob_not_found = 0
for i in range(dimension):
    for j in range(dimension):
        p = rand.random()
        if p < 0.25:
            cell_type = "flat"
            prob_not_found = 0.1
        elif p <0.50:
            cell_type = "hilly"
            prob_not_found = 0.3
        elif p <0.75:
            cell_type = "forested"
            prob_not_found = 0.7
        else:
            cell_type = "maze of caves"
            prob_not_found = 0.9
        env[i].append(Cell(cell_type, prob_not_found))

Enter dimension of board: 2


In [9]:
env[0][0].cell_type = "flat"
env[0][0].prob_not_found = 0.1
env[0][1].cell_type = "hilly"
env[0][1].prob_not_found = 0.3
env[1][0].cell_type = "forested"
env[1][0].prob_not_found = 0.7
env[1][0].target = 1
env[1][1].cell_type = "maze of caves"
env[1][1].prob_not_found = 0.9

In [44]:
belief = np.zeros((dimension,dimension))
for i in range(dimension):
    for j in range(dimension):
        belief[i][j] = 1/(dimension*dimension)
print(belief)

[[0.25 0.25]
 [0.25 0.25]]


In [19]:
def search_cell(current_position):
    x = current_position[0]
    y = current_position[1]
    
    if env[x][y].target == 1:
        p = rand.random()
        if p > env[x][y].prob_not_found:
            return True
    return False

In [24]:
#variable assignment tree
def marginalization(partial_query, numerator):
    probability = 1
    current = []
    if numerator:
        current = partial_query.pop()
    for item in partial_query:
        x = int(item[0])
        y = int(item[1])
        
        probability *= (1-belief[x][y])
        
    if numerator:
        probability *= belief[current[0]][current[1]]
    return probability    

In [36]:
def get_valid_neighbors(current_node):
  get_neighbors = [] #list of possible neighbors
  for i in range(-1,2):
    for j in range(-1,2):
      if i==0 and j==0:
        continue
      get_neighbors.append((i,j))
  x, y = current_node
  neighbors = set([])
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    neighbors.add((neighbor_x, neighbor_y))
  return neighbors

In [45]:
searching = True
remaining_spaces = dimension**2
current_position = [0,0]
times_searched = {}
while searching:
    print(f"current_position: {current_position}")
    remaining_spaces -=1
    #search cell
    query = search_cell(current_position)
    if query:
        print(f"Found target at {current_position}")
        searching = False
        break
    x = current_position[0]
    y = current_position[1]
    temp = {}
    temp[(x,y)] =1
    times_searched.update(temp)

    belief[x][y] *= env[x][y].prob_not_found
    temp_prob = 0
    for i in range(dimension):
        for j in range(dimension):
            if (i,j)  in times_searched:
                temp_prob += belief[i][j]
    for i in range(dimension):
        for j in range(dimension):
            if (i,j) not in times_searched:
                belief[i][j] = (1-temp_prob)/remaining_spaces
    print(belief)
    denominator = []
    numerator = []
    for key in times_searched:
        i = times_searched[key]
        for j in range(i):
            denominator.append(key)
            numerator.append(key)
    print("denominator")
    print(denominator)
    
    for i in range(dimension):
        for j in range(dimension):
            numerator.append((i,j))
            print("numerator")
            print(numerator)
            
            #Bayes Theorem
            prob_num = marginalization(numerator, True)#partial query of numerator
            prob_denom = marginalization(denominator, False)#partial query denominator
            print(f"prob_num: {prob_num}")
            print(f"prob_denom: {prob_denom}")
            belief[i][j] = prob_num/prob_denom #update belief
    print(belief)
    
    #choose next position
    max_belief = 0
    for i in get_valid_neighbors(current_position):
        if belief[i[0]][i[1]] > max_belief:
            max_belief = belief[i[0]][i[1]]
            current_position = i

current_position: [0, 0]
[[0.025 0.325]
 [0.325 0.325]]
denominator
[(0, 0)]
numerator
[(0, 0), (0, 0)]
prob_num: 0.024375
prob_denom: 0.975
numerator
[(0, 0), (0, 1)]
prob_num: 0.316875
prob_denom: 0.975
numerator
[(0, 0), (1, 0)]
prob_num: 0.316875
prob_denom: 0.975
numerator
[(0, 0), (1, 1)]
prob_num: 0.316875
prob_denom: 0.975
[[0.025 0.325]
 [0.325 0.325]]
current_position: (0, 1)
[[0.025   0.0975 ]
 [0.43875 0.43875]]
denominator
[(0, 0), (0, 1)]
numerator
[(0, 0), (0, 1), (0, 0)]
prob_num: 0.0219984375
prob_denom: 0.8799374999999999
numerator
[(0, 0), (0, 1), (0, 1)]
prob_num: 0.08579390625
prob_denom: 0.8799374999999999
numerator
[(0, 0), (0, 1), (1, 0)]
prob_num: 0.38607257812499995
prob_denom: 0.8799374999999999
numerator
[(0, 0), (0, 1), (1, 1)]
prob_num: 0.38607257812499995
prob_denom: 0.8799374999999999
[[0.025   0.0975 ]
 [0.43875 0.43875]]
current_position: (1, 0)
[[0.025    0.0975  ]
 [0.307125 0.570375]]
denominator
[(0, 0), (0, 1), (1, 0)]
numerator
[(0, 0), (0, 1), (